### Project 02: Create a GUI Notebook Program

Caleb Nebiyu - April 9, 2024

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [ ]:
# imports
import tkinter as tk
from tkinter import ttk
import datetime # one module for working with dates and times
from tkinter import filedialog

# The MainWindow class creates a custom GUI window based on the tkinter window (tk.Tk)
# It has an __init__() method, and three additional methods (new_note(), open_notebook(), and save_notebook())
# These methods correspond to new, open, and save buttons in the window.
# The new_note method calls the NoteForm class to create a new note form top level window.

class MainWindow(tk.Tk):
    def __init__(self):  #initialize the main window
        super().__init__() # initialize it as a tkinter window
        
        self.geometry("600x400") # set the default window size
        self.title('Notebook') #set the default window title
        self.notebook = [] # initialize an attribute named 'notebook' as an empty list
        self.notes = []        

        new_note_button = tk.Button(self, text = "New", command = self.new_note)
        new_note_button.pack()
        save_note_button = tk.Button(self, text = "Save", command = self.save_notebook)
        save_note_button.pack()
        open_note_button = tk.Button(self, text = "Open", command = self.open_notebook)
        open_note_button.pack()
        
        
    def new_note(self):
        note_window = NoteForm(self, self.notebook, self.notes)
        lorem = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Etiam sit amet suscipit mi, non porttitor mauris. Aliquam in lorem risus. Proin mauris mauris, varius ac vulputate sed, tempor nec lacus. Morbi sodales turpis in placerat semper. Donec bibendum blandit ante sit amet hendrerit.'
        note_window.title_entry.insert(0, 'New note title')
        note_window.text_entry.insert(tk.END, lorem)
        return None

    def open_notebook(self):
        filepath = filedialog.askopenfilename(filetypes = [("Text files", "*.txt"), ("All files", "*.*")])
        content_list = []
        if filepath:
            with open(filepath, "r") as file:
                for line in file:
                    content = line.split('\n')
                    content_list.extend(content)
                print(content_list)
                title = content_list[0].replace('Title: ', '').strip()
                text = content_list[2].replace('Text: ', '').strip()
                created = content_list[4].replace('Created: ', '').strip()
                note_dict = {'title': title, 'text': text, 'created': created}
                new_note = MakeNote(note_dict)
                print("Notebook Opened:", new_note) 
                self.note_button(new_note)
        return None
    
    def save_notebook(self):
        filepath = filedialog.asksaveasfilename(defaultextension = ".txt", filetypes = [("Text files", "*.txt"), ("All files", "*.*")])
        if filepath:
            with open(filepath, "w") as file:
                for note in self.notes:
                    filenote = f"Title: {note.title} \n Text: {note.text} \n Created: {note.created} \n"
                    file.write(filenote)  # Write each note to file
                    print(note.text)
            print("Notebook Saved Successfully")
        return None

    def note_window(self, new_note):
        window = tk.Toplevel(self)
        window.title("Note Details")
        label_new_note = tk.Label(window, text = f"Title: {new_note.title} \n Text: {new_note.text} \n Created: {new_note.created}")
        label_new_note.pack()
    
    def note_button(self, new_note):  
        button = tk.Button(self, text = new_note.title, command = lambda: self.note_window(new_note))
        button.pack()
    
# the NoteForm() class creates a Toplevel window that is a note form containing fields for
# data entry for title, text, link, and tags. It also calculates a meta field with date, time, and timezone
# the Noteform class has an __init__() method, and a submit() method that is called by a submit button
# the class may contain additional methods to perform tasks like calculating the metadata, for example
# the submit method calls the MakeNote class that transforms the the entered data into a new note object.

class NoteForm(tk.Toplevel):
    
    def __init__(self, master, notebook, notes): # initialize the new object
        super().__init__(master) # initialize it as a toplevel window

        # add additional lines to set the new window's attributes and default parameters

        self.master = master
        self.title("New Note")
        self.notebook = notebook
        self.notes = notes

        tk.Label(self, text="Note Title:").pack()
        self.title_entry = tk.Entry(self)
        self.title_entry.pack()

        tk.Label(self, text="Note Text:").pack()
        self.text_entry = tk.Text(self, height=5, width=40)
        self.text_entry.pack()

        tk.Button(self, text = "Submit", command=self.submit).pack()
        
    def submit(self):
        
        title = self.title_entry.get()
        text = self.text_entry.get("1.0", tk.END).strip()
        
        created = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        local_tz = datetime.datetime.now().astimezone().tzinfo
        meta = f"Created: {created}, {local_tz}"
        
        note_dict = {'title': title, 'text': text, 'created': created}
        
        self.notebook.append(note_dict)
        print("New note added:", note_dict)
        
        note_object = MakeNote(note_dict)
        self.notes.append(note_object)
        
        self.destroy()
        return None
        
        new_note = MakeNote(note_dict)
        self.notes.append(new_note)
        print(self.notes)
        return None

    
# The MakeNote class takes a dictionary containing the data entered into the form window,
# and transforms it into a new note object.
# At present the note objects have attributes but no methods.

class MakeNote():
    def __init__(self, note_dict):
        self.title = note_dict['title']
        self.text = note_dict['text']
        self.created = note_dict['created']
    
# main execution

if __name__ == '__main__':
    main_window = MainWindow() # this creates a notebook / main window called main_window. You may change the name if you want
    main_window.mainloop()

#### Print your three notes below

In [ ]:
# print your notes here
print("All notes:")
for note in main_window.notebook:
    print(note)